In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

from datetime import datetime, timedelta
import warnings

from minio_extractor import fetch_logs, RequestConfig

# Disable warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#For widescreen view
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 

spark = SparkSession \
    .builder \
    .appName("Spark_EID8_10") \
    .config("spark.sql.caseSensitive","True") \
    .config('spark.sql.execution.arrow.pyspark.enabled', True) \
    .config('spark.sql.session.timeZone', 'EST') \
    .config('spark.driver.memory','16G') \
    .config('spark.ui.showConsoleProgress', True) \
    .config('spark.sql.repl.eagerEval.enabled', True) \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

### reading from a parquet object
park = spark.read.parquet("edr_all_raw.parquet")

In [ ]:
park.printSchema()
park.show(n = 1, vertical = True)
park.createOrReplaceTempView('sparktable')

In [ ]:
EID8 = spark.sql(
'''
SELECT  UtcTime, computer_name, SourceProcessGuid, SourceImage, TargetProcessGuid, TargetImage, StartAddress, StartFunction, StartModule, SourceUser
FROM sparktable b
WHERE b.event_id = 8 AND source_name = "Microsoft-Windows-Sysmon"
'''
).show(10,False)

In [ ]:
EID10 = spark.sql(
'''
SELECT  UtcTime, computer_name, SourceProcessGUID, SourceImage, TargetProcessGUID, TargetImage, GrantedAccess, SourceUser
FROM sparktable a
WHERE a.event_id = 10 AND source_name = "Microsoft-Windows-Sysmon"
AND (a.GrantedAccess) == '0x1400'
'''
).show(10,False)

In [ ]:
ReflectiveDLL_ProcessInjection = spark.sql(
'''
SELECT b.UtcTime, b.computer_name,b.SourceImage, b.TargetImage, b.StartAddress, b.StartFunction, b.StartModule, b.SourceUser, a.SourceProcessGUID, a.TargetProcessGUID, a.GrantedAccess
FROM sparktable b
JOIN sparktable a
ON a.computer_name = b.computer_name
AND a.event_id = 10 AND a.source_name = "Microsoft-Windows-Sysmon"
AND ((a.GrantedAccess) == '0x1f3fff' OR (a.GrantedAccess) == '0x147a')
WHERE b.event_id = 8 AND b.source_name = "Microsoft-Windows-Sysmon"
AND NOT
    (
        lower(b.SourceImage) LIKE '%dwm.exe'
        OR lower(b.SourceImage) LIKE '%csrss.exe'
        OR b.SourceUser == "Window Manager\DWM-2"
    )

'''
).show(10,False)